In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from collections import Counter
df = pd.read_csv("lego.population.csv", sep = ",", encoding = "latin1")


# Map the theme Genderegory to each row based on the 'Theme' column
#unique_themes = set(data['Theme'].unique())
#data['Gender_Genderegory'] = data['Theme'].map(theme_Genderegories_updated)

Df
# fjerner forklaringsvariabler vi ikke trenger
df2 = df[['Set_Name', 'Theme', 'Weight', 'Price']]

# fjerner observasjoner med manglende datapunkter
df2 = df2.dropna()

# gjør themes om til string og fjern alle tegn vi ikke vil ha med
df2['Theme'] = df2['Theme'].astype(str)
df2['Theme'] = df2['Theme'].str.replace(r'[^a-zA-Z0-9\s-]', '', regex = True)

# fjerner dollartegn og trademark-tegn fra datasettet
df2['Price'] = df2['Price'].str.replace('\$', '', regex = True)

# og gjør så prisen om til float
df2['Price'] = df2['Price'].astype(float)



# Creating a dictionary to map themes to theme Genderegories
theme_Genderegories_updated = {
    "Architecture": "Nøytral", "Batman": "Gutt", "BrickHeadz": "Nøytral",
    "City": "Jente", "Classic": "Nøytral", "Creator 3-in-1": "Nøytral",
    "Creator Expert": "Nøytral", "DC": "Gutt", "Disney": "Jente", "DOTS": "Jente",
    "DUPLO": "Nøytral", "Friends": "Jente", "Harry Potter": "Nøytral",
    "Hidden Side": "Gutt", "Ideas": "Nøytral", "Juniors": "Nøytral",
    "Jurassic World": "Gutt", "LEGO Art": "Nøytral", "LEGO Brick Sketches": "Nøytral",
    "LEGO EduGenderion": "Nøytral", "LEGO Frozen 2": "Jente", "LEGO Super Mario": "Nøytral",
    "Marvel": "Gutt", "Minecraft": "Nøytral", "Minifigures": "Nøytral",
    "Minions": "Nøytral", "Monkie Kid": "Gutt", "NINJAGO": "Gutt", "Overwatch": "Gutt",
    "Powered UP": "Nøytral", "Powerpuff Girls": "Jente", "Speed Champions": "Gutt",
    "Spider-Man": "Gutt", "Star Wars": "Gutt", "Stranger Things": "Nøytral",
    "Technic": "Gutt", "THE LEGO MOVIE 2": "Nøytral", "Trolls World Tour": "Jente",
    "Unikitty": "Jente", "Xtra": "Nøytral"
}


# Mapping the "Theme" column to the new "Theme_Genderegory" column based on theme_Genderegories_updated
df2['Gender'] = df2['Theme'].map(theme_Genderegories_updated).fillna('Unknown')

def extract_weight_in_grams(weight_str):
    if pd.isna(weight_str):
        return None
    weight_kg = float(weight_str.split("Kg")[0])  # Extract numeric part in kg
    return weight_kg * 1000  # Convert kg to grams

# Applying the function to the "Weight" column
df2['vekt'] = df2['Weight'].apply(extract_weight_in_grams)

# det er dataset dere skal bruke!
df2.to_csv('rensetdata.csv')
df2
# Summing up the number of entries for each Theme_Genderegory
theme_Genderegory_counts = df2['Gender'].value_counts()

# Displaying the result to the user
theme_Genderegory_counts
farge=['#1f77b4', '#ff7f0e', '#2ca02c']
sns.boxplot(x='Gender', y='Price', data=df2, palette=farge)

# Tillegg titler og akselabeler for klarhet
plt.title('Prisfordeling på Kjønn')
plt.xlabel('Kjønn')
plt.ylabel('Pris')

# Viser plottet
plt.show()

# Statistikk for hver enkelt kategori
stats = df2.groupby('Gender')['Price'].describe()
print(stats)
# informasjon for Kjønn og pris
df_Gutt1 = df2[df2['Gender'] == 'Nøytral']
resultat_Gutt1 = smf.ols('Price ~ Gender', data = df_Gutt1).fit()

resultat_Gutt1.summary()
# informasjon for Kjønn og pris
df_Gutt1 = df2[df2['Gender'] == 'Jente']
resultat_Gutt1 = smf.ols('Price ~ Gender', data = df_Gutt1).fit()

resultat_Gutt1.summary()
slope = resultat_lr.params['vekt']
intercept = resultat_lr.params['Intercept']

regression_x = np.array(df2['vekt'])

regression_y = slope * regression_x + intercept

plt.scatter(df2['vekt'], df2['Price'], label='Data Points')
plt.plot(regression_x, regression_y, color='red', label='Regression Line')
plt.xlabel('Vekt i gram')
plt.ylabel('Pris i $')
plt.title('Kryssplott med regresjonslinje (enkel LR)')
plt.legend()
plt.grid()
plt.show()
figure, axis = plt.subplots(1, 2, figsize = (15, 5))
sns.scatterplot(x = resultat_lr.fittedvalues, y = resultat_lr.resid, ax = axis[0])
axis[0].set_ylabel("Residual")
axis[0].set_xlabel("Predikert verdi")

sm.qqplot(resultat_lr.resid, line = '45', fit = True, ax = axis[1])
axis[1].set_ylabel("Kvantiler i residualene")
axis[1].set_xlabel("Kvantiler i normalfordelingen")
plt.show()

myGender = ['Gutt', 'Jente', 'Nøytral']
intercept = [modell_Gender_mlr.params['Gender[T.Jente]'], modell_Gender_mlr.params['Gender[T.Nøytral]'], 0] + modell_Gender_mlr.params['Intercept']
slope = modell_Gender_mlr.params['vekt']

for i, Gender in enumerate(myGender):

    regression_x = np.array(df2[df2['Gender'].isin([Gender])]['vekt'])
    regression_y = slope * regression_x + intercept[i]

    # Plot scatter plot and regression line
    plt.scatter(df2[df2['Gender'].isin([Gender])]['vekt'], df2[df2['Gender'].isin([Gender])]['Price'], color = plt.cm.tab10(i))
    plt.plot(regression_x, regression_y, color=plt.cm.tab10(i), label=Gender)

# uten Gender som forklaringsvariabel:
regression_x = np.array(df2['vekt'])
regression_y = res_sub.params['vekt'] * regression_x + res_sub.params['Intercept']
plt.plot(regression_x, regression_y, color='black', label = 'Kjønn ikke tatt hensyn til')
    
plt.xlabel('Vekt i gram')
plt.ylabel('Pris i $')
plt.title('Kryssplott med regresjonslinjer')
plt.legend()
plt.grid()
plt.show()

